In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
import os
import sys
import math
import warnings
from geopy.geocoders import Nominatim
import pyproj

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
cwd = os.getcwd()

education_folder_path = os.path.dirname(cwd)

sys.path.append(education_folder_path)

In [3]:
from global_functions import remove_spaces_in_columns, up_load_shp, up_load_df, drop_geo, change_cols_names, get_newest_date_file, get_output_folder_location

In [4]:
#בתי ספר וגנים מעיריית ירושלים
JLM=up_load_df(r'{}\background_files\jerusalem_muni'.format(education_folder_path),'מוסדות בירושלים 2020')
JLM = remove_spaces_in_columns(JLM)

In [5]:
JLM.rename(columns={'סמל_חינוך': 'סמל_מוסד'}, inplace=True)

JLM = JLM.drop(columns=['אגף', 'פיקוח', 'סוג_חינוך', 'מעמד_משפטי', 'מספר_כיתות', "מס'_תלמידים_ז-ט", "מס'_תלמידים_י-יד", 'קוד_אזור_סטיסטי', 'תאור_אזור_סטטיסטי', 'Unnamed:_18', 'Unnamed:_19'])

JLM.head()

,סמל_עירייה,סמל_מוסד,שם_מוסד,שלב_חינוך,מכיתה,עד_כיתה,"סה""כ_תלמידים",coordinate_x,coordinate_y
0,21321.00,110106.00,"מ""מ אגרון",יסודי,א,ו,230.00,216876.00,629987.00
1,21323.00,110312.00,"מ""מ ארגנטינה הניסויי",יסודי,א,ח,347.00,216154.00,630603.00
2,21326.00,110155.00,"מ""מ בית הכרם -הישראלי",יסודי,א,ו,534.00,218166.00,632109.00
3,21327.00,110171.00,"מ""מ תל""י גאולים ע""ש יצחק נבון",יסודי,א,ו,331.00,220805.00,629266.00
4,21328.00,111112.00,"מ""מ זלמן ארן",יסודי,א,ו,367.00,220633.00,628408.00


In [6]:
# פונקציה להמרת כיתות לאותיות למספרים
def get_grade_number(grade):
    grade_map = {
        'א': 1, 'ב': 2, 'ג': 3, 'ד': 4, 'ה': 5, 'ו': 6,
        'ז': 7, 'ח': 8, 'ט': 9, 'י': 10, 'יא': 11, 'יב': 12, 
        'יג': 13, 'יד': 14
    }
    return grade_map.get(grade, np.nan)

In [7]:
# חישוב מספר הכיתות
JLM['מספר_כיתות'] = JLM.apply(lambda row: get_grade_number(row['עד_כיתה']) - get_grade_number(row['מכיתה']) + 1, axis=1)
JLM['מספר_כיתות']

# חלוקת התלמידים לפי כיתות
JLM['תלמידים_לכיתה'] = JLM['סה"כ_תלמידים'] / JLM['מספר_כיתות']


In [8]:
# יצירת עמודות עבור כל כיתה
grades = ['א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'יא', 'יב', 'יג', 'יד']
for grade in grades:
    JLM[grade] = JLM.apply(lambda row: row['תלמידים_לכיתה'] if get_grade_number(row['מכיתה']) <= get_grade_number(grade) <= get_grade_number(row['עד_כיתה']) else 0, axis=1)

In [9]:
# פונקציה להמרת כיתות לאותיות למספרים
def get_grade_range_number(grade):
    grade_map = {
        'א': 1, 'ב': 2, 'ג': 3, 'ד': 4, 'ה': 5, 'ו': 6,
        'ז': 7, 'ח': 8, 'ט': 9, 'י': 10, 'יא': 11, 'יב': 12, 'יג': 13, 'יד': 14,
        '3': 3, '4': 4, '5': 5  # עבור גני ילדים
    }
    return grade_map.get(grade, np.nan)

In [10]:
# פונקציה לסיווג לפי שכבת גיל
def classify_students(row, grades_dict):
    # אם זה גני ילדים, נכניס את כל התלמידים לקטגוריה kgarden
    if row['שלב_חינוך'] == 'גני ילדים':
        return pd.Series({'kgarden_stu': row['סה"כ_תלמידים'], 'ele_stu': 0, 'mid_stu': 0, 'high_stu': 0})
    
    # חישוב מספר הכיתות בטווח
    from_grade = get_grade_range_number(row['מכיתה'])
    to_grade = get_grade_range_number(row['עד_כיתה'])
    
    # חלוקה לפי שכבות גיל
    ele_students = 0
    mid_students = 0
    high_students = 0

    # נבדוק באיזה טווח כיתות הערכים ונחלק את התלמידים לפי שכבת גיל
    if from_grade <= 6 and to_grade >= 1:  # יסודי
        ele_students = row['סה"כ_תלמידים'] * len([g for g in grades_dict['ele'] if from_grade <= get_grade_range_number(g) <= to_grade]) / (to_grade - from_grade + 1)
    
    if from_grade <= 9 and to_grade >= 7:  # חטיבת ביניים
        mid_students = row['סה"כ_תלמידים'] * len([g for g in grades_dict['mid'] if from_grade <= get_grade_range_number(g) <= to_grade]) / (to_grade - from_grade + 1)
    
    if from_grade >= 10:  # תיכון
        high_students = row['סה"כ_תלמידים'] * len([g for g in grades_dict['high'] if from_grade <= get_grade_range_number(g) <= to_grade]) / (to_grade - from_grade + 1)
    
    return pd.Series({'kgarden_stu': 0, 'ele_stu': ele_students, 'mid_stu': mid_students, 'high_stu': high_students})

In [11]:
# המרת תלמידים לפי שכבות הגיל שהוגדרו
ages = {'kgarden':[3, 4, 5],'ele':['א', 'ב', 'ג', 'ד','ה','ו'],'mid':['ז', 'ח', 'ט'], 'high':['י', 'יא', 'יב', 'יג', 'יד']}

# הוספת עמודות של התלמידים לפי שכבת גיל
JLM[['kgarden_stu', 'ele_stu', 'mid_stu', 'high_stu']] = JLM.apply(lambda row: classify_students(row, ages), axis=1)

# תצוגת הדאטה פריים הסופי
JLM.head()


,סמל_עירייה,סמל_מוסד,שם_מוסד,שלב_חינוך,מכיתה,עד_כיתה,"סה""כ_תלמידים",coordinate_x,coordinate_y,מספר_כיתות,תלמידים_לכיתה,א,ב,ג,ד,ה,ו,ז,ח,ט,י,יא,יב,יג,יד,kgarden_stu,ele_stu,mid_stu,high_stu
0,21321.00,110106.00,"מ""מ אגרון",יסודי,א,ו,230.00,216876.00,629987.00,6.00,38.33,38.33,38.33,38.33,38.33,38.33,38.33,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,230.00,0.00,0.00
1,21323.00,110312.00,"מ""מ ארגנטינה הניסויי",יסודי,א,ח,347.00,216154.00,630603.00,8.00,43.38,43.38,43.38,43.38,43.38,43.38,43.38,43.38,43.38,0.00,0.00,0.00,0.00,0.00,0.00,0.00,260.25,86.75,0.00
2,21326.00,110155.00,"מ""מ בית הכרם -הישראלי",יסודי,א,ו,534.00,218166.00,632109.00,6.00,89.00,89.00,89.00,89.00,89.00,89.00,89.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,534.00,0.00,0.00
3,21327.00,110171.00,"מ""מ תל""י גאולים ע""ש יצחק נבון",יסודי,א,ו,331.00,220805.00,629266.00,6.00,55.17,55.17,55.17,55.17,55.17,55.17,55.17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,331.00,0.00,0.00
4,21328.00,111112.00,"מ""מ זלמן ארן",יסודי,א,ו,367.00,220633.00,628408.00,6.00,61.17,61.17,61.17,61.17,61.17,61.17,61.17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,367.00,0.00,0.00
5,21331.00,111351.00,"מ""מ יאנוש קורצ'אק",יסודי,א,ו,242.00,222809.00,634375.00,6.00,40.33,40.33,40.33,40.33,40.33,40.33,40.33,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,242.00,0.00,0.00
6,21332.00,111682.00,"מ""מ תל""י בית וגן",יסודי,א,ו,426.00,216942.00,629444.00,6.00,71.00,71.00,71.00,71.00,71.00,71.00,71.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,426.00,0.00,0.00
7,21333.00,111203.00,"מ""מ בי'ס משלב גבעת גונן",יסודי,א,ו,157.00,218602.00,629117.00,6.00,26.17,26.17,26.17,26.17,26.17,26.17,26.17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,157.00,0.00,0.00
8,21334.00,111500.00,"מ""מ גילה א' ללימודי סביבה וחברה",יסודי,א,ו,457.00,218951.00,626817.00,6.00,76.17,76.17,76.17,76.17,76.17,76.17,76.17,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,457.00,0.00,0.00
9,21335.00,111294.00,"מ""מ חטיבה צעירה דוד ילין",יסודי,א,ב,60.00,218077.00,632176.00,2.00,30.00,30.00,30.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,60.00,0.00,0.00
